# Exploitation Zone - Model Predictiu

- exploitation zone del model predictiu
- preparation pipeline per taula d'entrenament del model --> cada zipcode és un indiv
    - Sales: 5 categories més comunes per zipcode, count vendes per zipcode, profit mitja per zipcode, mitjana num unitat per comanda per zipcode
    - Shops: 5shops més comunes per zipcode
    - Income: mitjana income per zipcode